In [1]:
# %matplotlib notebook

from IPython.display import HTML

HTML('''
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>
''')

# Introduction

In this activity, we'll be using satellite imagery to automatically determine how the water level of a particular lake changes over time. We can then use this to predict what will happen to the lake in the future! 

In order to do this, we need to develop a system that will detect water in an image.

# Colour Selection

Our satellite images are colour images. All colour images consist of three components: red, green and blue. The first step is to select which colours (or ratio of colours) we want to work with.

Click on the buttons below to select which colours you want. The graph on the left is the original image, while the one on the right is the colour component you selected (note that the individual colour components render in greyscale!) 

Think about the three colours and how they relate to land and water. We want an image that will separate the water from the land as best as possible. But how do we know which one is best?

The histogram graph below shows the magnitude frequency of the new image. **The best choices will result in a graph with two peaks (a bimodal graph), which correspond to land and water respectively!**   

Once you've made your choice, move on to the next step.

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage.color import label2rgb

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import data
from skimage import io
from skimage import color
from IPython.display import HTML

global image_rgb
global image
global image_cleaned
global threshold_value
global choice

choice = "All Bands"
threshold_value = 0
image_rgb = io.imread('1984.png')
image = image_rgb
training_years = range(1984, 2008)
testing_years = range(2008, 2030) 

import warnings
warnings.filterwarnings("ignore")
 
def _select_bands(image_rgb, choice):
    if choice == "All Bands":
        image = color.rgb2grey(image_rgb) * 255
    elif choice == "Red Band":
        image = image_rgb[:, :, 0] * 255
    elif choice == "Green Band":
        image = image_rgb[:, :, 1] * 255
    elif choice == "Blue Band":
        image = image_rgb[:, :, 2] * 255
    elif choice == "Red-Green Ratio":
        image = image_rgb[:, :, 0] / image_rgb[:, :, 1]
        image = image / image.max() * 255   
    elif choice == "Red-Blue Ratio":
        image = image_rgb[:, :, 0] / image_rgb[:, :, 2]
        image = image / image.max() * 255   
    elif choice == "Green-Red Ratio":
        image = image_rgb[:, :, 1] / image_rgb[:, :, 0]
        image = image / image.max() * 255   
    elif choice == "Green-Blue Ratio":
        image = image_rgb[:, :, 1] / image_rgb[:, :, 2]
        image = image / image.max() * 255   
    elif choice == "Blue-Red Ratio":
        image = image_rgb[:, :, 2] / image_rgb[:, :, 0]
        image = image / image.max() * 255   
    elif choice == "Blue-Green Ratio":
        image = image_rgb[:, :, 2] / image_rgb[:, :, 1]
        image = image / image.max() * 255   
    else:
        raise ValueError
    return image
    
    



def segment_image(image_rgb, spectrum, threshold):
    """
    Return the segmented image and area
    """
    im = _select_bands(image_rgb, spectrum)
    clean_im = morphology.remove_small_objects((im < threshold), 50)
    segmentation = ndi.binary_fill_holes(clean_im)
    labeled_image, _ = ndi.label(segmentation)
    image_label_overlay = label2rgb(labeled_image, image=clean_im)
    plt.imshow(im_rgb, cmap=plt.cm.gray, interpolation='nearest')
    plt.contour(segmentation, [0.5], linewidths=1.2, colors='r')
    return plt, image_label_overlay[:,:,1].sum()


def _calculate_area(clean_im):
    segmentation = ndi.binary_fill_holes(clean_im)
    labeled_image, _ = ndi.label(segmentation)
    image_label_overlay = label2rgb(labeled_image, image=clean_im)
    return image_label_overlay[:,:,1].sum()

    

In [3]:
def select_bands(c):
    
    global image
    global image_rgb
    global choice
    choice = c
    image = _select_bands(image_rgb, c)
    

    fig = plt.figure(figsize=(12, 10))
    ax1 = fig.add_subplot(2,2,1)
    ax1.set_title("Original Image")
    ax1.imshow(image_rgb, cmap=plt.cm.gray, interpolation='nearest')
    ax2 = fig.add_subplot(2,2,2)
    ax2.set_title("New Image")
    ax2.imshow(image, cmap=plt.cm.gray, interpolation='nearest')
    ax3 = fig.add_subplot(2,2,3)
    hist = np.histogram(image, bins=np.arange(0, 256))
    ax3.set_title("Histogram")
   
    plt.plot(hist[1][:-1], hist[0], lw=2)
    
    
    #plt.imshow(image_rgb, cmap=plt.cm.gray, interpolation='nearest')
    #plt.show()
    #plt.imshow()
    #plt.show()
    ##hist = np.histogram(image, bins=np.arange(0, 256))
    #plt.plot(hist[1][:-1], hist[0], lw=2)
    #plt.show()


buttons = widgets.ToggleButtons(
    options=['All Bands', 'Red Band', 'Green Band', 'Blue Band', 'Red-Green Ratio', 'Red-Blue Ratio', 'Green-Red Ratio', 'Green-Blue Ratio','Blue-Red Ratio', 'Blue-Green Ratio'],
    description='Colours:',
    disabled=False,
    selected=2,
    button_style='',
)

interact(select_bands, c=buttons);

interactive(children=(ToggleButtons(description='Colours:', options=('All Bands', 'Red Band', 'Green Band', 'Blue Band', 'Red-Green Ratio', 'Red-Blue Ratio', 'Green-Red Ratio', 'Green-Blue Ratio', 'Blue-Red Ratio', 'Blue-Green Ratio'), value='All Bands'), Output()), _dom_classes=('widget-interact',))

## Note:

Why didn't selecting blue work at all? Lakes are blue, right?! Well, that's because the land, which is a bit greyish, also contains blue. Additionally, plantlife is green, which is quite similar to blue. Red, on the other hand, has absolutely nothing at all to do with water! And thus it's a good choice of colour, since the **absence** of red is a good indication of water!

# Thresholding

We now pick which sections of the image are part of the lake, and which are part of the land. Drag the slider to select a value. All pixels with values greater than the threshold will be considered as the lake. Everything else is the land.

**Use the histogram above to select the threshold by picking a value in between the two peaks!**

Try pick the value so the image looks like the shape of the lake (remember that the lake has two islands!). Once you're satisfied, move to the next step. If the thresholding doesn't result in a lake-shaped image, you may have to pick different colours.

In [4]:

def threshold(val):
    global threshold_value
    threshold_value = val
    
    fig = plt.figure(figsize=(9, 9))
    ax = fig.add_subplot(1,1,1)
    ax.set_title("Image after Thresholding")
    ax.imshow(image < val, cmap=plt.cm.gray, interpolation='nearest')
    
    #plt.imshow(image > val, cmap=plt.cm.gray, interpolation='nearest')
    
    
    
    #plt.show()
    #plt.tight_layout()

threshold_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=255,
    step=1,
    description='Threshold:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

interact(threshold, val=threshold_slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Threshold:', max=255), Output()), _dom_classes=('widget-interact',))

<function __main__.threshold>

# Remove small objects

There may be small specks in the above image. To remove these artefacts, click on the button below which removes shapes smaller than a certain size. This will clean up the image a bit.

In [5]:
from skimage import morphology


def remove_small_objects(button):
    from IPython import display as dsp
    dsp.clear_output(wait=True)
    display(button)
    global image
    global threshold_value
    global image_cleaned
    image_cleaned = morphology.remove_small_objects((image < threshold_value), 50)
    fig = plt.figure(figsize=(9, 9))
    ax = fig.add_subplot(1,1,1)
    ax.set_title("Cleaned Image")
    
    # fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(image_cleaned, cmap=plt.
              cm.gray, interpolation='nearest')
    ax.axis('off')
    
    #ax.set_adjustable('box-forced')


button1 = widgets.Button(description="Click to clean image")
display(button1)
button1.on_click(remove_small_objects)


Button(description='Click to clean image', style=ButtonStyle())

# Segmentation

Given the above image, we now find different connected clusters in the image, which we then label with different colours. Looking at the original images, we can expect three clusters: the land, the lake and the islands.

Click on the button below to segment the image into these different classes.

The left image shows the different classes. If everything has gone to plan, the right image should provide a nice outline of the lake.

If you're happy with the outline, move to the next step.


In [6]:
def segment(button):
    from IPython import display as dsp
    dsp.clear_output(wait=True)
    display(button)
    global image
    global image_rgb
    global image_cleaned
    segmentation = ndi.binary_fill_holes(image_cleaned)
    labeled_image, _ = ndi.label(segmentation)
    
    image_label_overlay = label2rgb(labeled_image, image=image_cleaned)
    # print(image_label_overlay[:,:,1].sum())  
    
    fig = plt.figure(figsize=(12, 10))
    ax1 = fig.add_subplot(1,2,2)
    
    ax1.imshow(image_rgb, cmap=plt.cm.gray, interpolation='nearest')
    ax1.contour(segmentation, [0.5], linewidths=1.2, colors='r')
    ax1.set_title("Contour Overlay")
    
    ax2 = fig.add_subplot(1,2,1)
    ax2.imshow(image_label_overlay, interpolation='nearest')
    ax2.set_title("Segmentation")
    #plt.show()
    #plt.imshow(image_label_overlay, interpolation='nearest')
    plt.tight_layout()
    
    
button2 = widgets.Button(description="Click to segment")
display(button2)
button2.on_click(segment)


Button(description='Click to segment', style=ButtonStyle())

# Apply to Multiple Images

We have satellite images from a number of consecutive years. So let's apply your colour choice and threshold to these images too. Click on the button to generate an animation of these images. Be careful! What worked for the first image may not work for the rest of them! If you're happy with the result, move to the final step. Otherwise, go back and change your colour or threshold choice. 

**If you make a change, you must execute all the steps again, including removing small objects and the segmentation**

In [11]:
from matplotlib import animation  
import matplotlib.image as mgimg
import imageio
from IPython.display import Image

def segment_image(year):
    
    global choice
    global threshold_value

    file = '{}.png'.format(year)
    im_rgb = io.imread(file)
    im = _select_bands(im_rgb, choice)
    
    clean_im = morphology.remove_small_objects((im < threshold_value), 50)
    segmentation = ndi.binary_fill_holes(clean_im)
    labeled_image, _ = ndi.label(segmentation)
    image_label_overlay = label2rgb(labeled_image, image=clean_im)
    
    plt.clf()
    plt.title(str(year))
    plt.imshow(im_rgb, cmap=plt.cm.gray, interpolation='nearest')
    plt.contour(segmentation, [0.5], linewidths=1.2, colors='r')
    fname = "temp_{}.png".format(year)
    plt.savefig(fname)   
    return fname


def build_animation(button):
    
    from IPython import display as dsp
    dsp.clear_output(wait=True)
    display(button)
    
    
    progress = widgets.IntProgress(
    value=0,
    min=0,
    max=24 + 1,
    step=1,
    description='Building:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
    )
    display(progress)
    
    
    #from IPython import display as dsp
    #dsp.clear_output(wait=True)
    #display(button)
        
    # segment_image(1984)
    #from IPython import display as dsp
    #dsp.clear_output(wait=True)
    #interact(segment_image, year=year_slider)
    files = []
    for i in range(1984, 2008):
        files.append(segment_image(i))
        progress.value += 1
    
    imageio.mimsave('temp.gif', [imageio.imread(f) for f in files], 'GIF', duration=0.5)
    progress.value += 1
    display(Image(filename='temp.gif', width=750, height=500))
    #ani = animation.ArtistAnimation(fig, ims, interval=1000, blit=True, repeat_delay=1000)
    #plt.show()
    plt.clf()

button3 = widgets.Button(description="Build Animation")
display(button3)

button3.on_click(build_animation)



Button(description='Build Animation', style=ButtonStyle())

# Model the Future

Once you've settled on your choices, we can now use your model to build a prediction for what will happen to the lake.

We do this by first calculating the area of the outlines in the above animation. We then try to fit a quadratic function through these points as best we can. 

The generated graph shows the calculated areas in blue, and the quadratic function in yellow. We can use the yellow line to estimate the trend in the amount of water in the lake and use it to make predictions.


In [8]:

def calculate_area(im, choice, threshold_value, progress=None):
    im_rgb = io.imread(im)
    im = _select_bands(im_rgb, choice)
   
    
    clean_im = morphology.remove_small_objects((im < threshold_value), 50)
    segmentation = ndi.binary_fill_holes(clean_im)
    labeled_image, _ = ndi.label(segmentation)
    image_label_overlay = label2rgb(labeled_image, image=clean_im)
    if progress is not None:
        progress.value += 1
    return image_label_overlay[:,:,1].sum()                      
    
def predict(button):
    global choice
    global threshold_value
    
    from IPython import display as dsp
    dsp.clear_output(wait=True)
    display(button)
    
    
    progress = widgets.IntProgress(
    value=0,
    min=0,
    max=24,
    step=1,
    description='Building:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
    )
    display(progress)
    
    
    x = np.array([i for i in range(1984, 2008)])
    vals = [calculate_area('{}.png'.format(i), choice, threshold_value, progress) for i in x]

    z = np.poly1d(np.polyfit(x, np.array(vals), 2))
    #plt.clf()
    fig = plt.figure(figsize=(8, 6))
    plt.plot(vals)
    plt.plot(z(x))
    plt.xlabel('Year')
    plt.ylabel('Total Area')
    plt.title('Lake Area Changes Over Time')
    plt.show()
    
    
button4 = widgets.Button(description="Build predictive model")
display(button4)
button4.on_click(predict)

#image_label_overlay[:,:,1].sum()

Button(description='Build predictive model', style=ButtonStyle())

# Visualise the Future

Now that we can predict the lake size in the future, we can visualise this. Click on the button below to see what happens to the lake if it continues to follow this trend. We make this prediction by *eroding* the lake at the specified rate.

In [9]:

def predict_image(future, image_rgb, spectrum, threshold, area):
    """
    Return the segmented image and area
    """
    im = _select_bands(image_rgb, spectrum)
    clean_im = morphology.remove_small_objects((im < threshold), 50)
        
    #plt.imshow(clean_im, cmap=plt.cm.gray, interpolation='nearest')
    
    
    #erode/dilate cleaned image until correct area is achieved
    current_area = _calculate_area(clean_im)
    current_im = clean_im
    while True:
        
        diff = current_area - area
        
        if diff > 0:
            #erode
            new_clean_im = ndi.binary_erosion(current_im)
        else:
            #dilate
            new_clean_im = ndi.binary_dilation(current_im)
            
        
        new_clean_im = morphology.remove_small_objects(new_clean_im, 50)
        
        #plt.imshow(new_clean_im, cmap=plt.cm.gray, interpolation='nearest') 
        #plt.show()
        
        new_area = _calculate_area(new_clean_im)
        new_diff = new_area - area
        if np.sign(diff) != np.sign(new_diff):
            # overshoot. Pick the better of the two
            if np.abs(new_diff) < np.abs(diff):
                clean_im = new_clean_im
            else:
                clean_im = current_im
            break
        else:
            # keep going
            current_im = new_clean_im
            current_area = new_area
    
    segmentation = ndi.binary_fill_holes(clean_im)
    labeled_image, _ = ndi.label(segmentation)
    image_label_overlay = label2rgb(labeled_image, image=clean_im)
    
    plt.clf()
    #fig = plt.figure(figsize=(8, 6))
    temp = "s" if future != 1 else ""
    plt.title("{} year{} in future".format(future, temp))
    plt.imshow(image_rgb, cmap=plt.cm.gray, interpolation='nearest')
    plt.contour(segmentation, [0.5], linewidths=1.2, colors='r')
    fname = "_temp_{}.png".format(future)
    plt.savefig(fname)   
    return fname, image_label_overlay[:,:,1].sum()               
    
def predict(button):
    global choice
    global threshold_value
    
    from IPython import display as dsp
    dsp.clear_output(wait=True)
    display(button)   
    
    
    progress2 = widgets.IntProgress(
    value=0,
    min=0,
    max=2030-2008,
    step=1,
    description='Animating:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
    )
    display(progress2)
    
    x = np.array([i for i in range(1984, 2008)])
    vals = [calculate_area('{}.png'.format(i), choice, threshold_value) for i in x]
    z = np.poly1d(np.polyfit(x, np.array(vals), 2))
    
    image_rgb = io.imread('2007.png')
    files = []
    

    
    for i, year in enumerate(testing_years):
        
        area = z(year)
        if area < 0:
            break
        x, area = predict_image(i, image_rgb, choice, threshold_value, area)
        files.append(x)
        progress2.value += 1
    progress2.value = 2030-2008
    plt.clf()
    imageio.mimsave('_temp.gif', [imageio.imread(f) for f in files], 'GIF', duration=0.5)
    display(Image(filename='_temp.gif', width=750, height=500))    
    
button5 = widgets.Button(description="Visualise predictions")
display(button5)
button5.on_click(predict)

#image_label_overlay[:,:,1].sum()

Button(description='Visualise predictions', style=ButtonStyle())

# Congrats!

You've just used machine learning to building a predictive model on satellite imagery :)

In [10]:

HTML('''
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
You can toggle the code on and off by clicking <a href="javascript:code_toggle()">here</a>.''')